In [1]:
import pandas as pd
import numpy as np

In [5]:
excel = pd.read_excel(r"D:\TDC.xlsx", sheet_name=None)

In [6]:
type(excel)

dict

In [7]:
excel.keys()

dict_keys(['Santander', 'HSBC', 'Nu', 'Hoja1', 'CETES', 'BBVA2', 'BBVA1', 'Libertad', 'CGV', 'Préstamos', 'Rafting', 'Allianz', 'Presuputesto Ago'])

In [8]:
excel_nu = excel['Nu']
type(excel_nu)

pandas.core.frame.DataFrame

In [13]:
excel_nu.columns = excel_nu.loc[0]

In [17]:
excel_nu.drop(0, axis=0, inplace=False).head(3)

,Fecha,Descripción,Depósito,Retiro,Depósito,Retiro,Depósito,Retiro,Depósito,Retiro,mensualidad,capital,gastos,fondo,total,NaN
1,2025-01-03 00:00:00,mensualidad,4500,NaN,NaN,NaN,3697.44,NaN,NaN,NaN,4500,0,3697.44,0,8197.44,NaN
2,2025-01-03 00:00:00,Ahorro-fondo,NaN,NaN,NaN,NaN,NaN,NaN,54368.05,NaN,4500,0,3697.44,54368.05,62565.49,NaN
3,2025-01-06 00:00:00,Retiro-mosquitero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4000,4500,0,3697.44,50368.05,58565.49,NaN


In [18]:
import nibabel as nib

ModuleNotFoundError: No module named 'nibabel'

# RMI

https://openneuro.org/search/modality/mri?query=%7B%22modality_selected%22%3A%22mri%22%2C%22keywords%22%3A%5B%22ds000117%22%5D%7D

In [ ]:
import nibabel as nib
import numpy as np

# Ruta al archivo NIfTI
ruta = "ruta/a/tu_archivo.nii.gz"

# Cargar con nibabel
img = nib.load(ruta)
data = img.get_fdata()

print("Dimensiones:", data.shape)
print("Tipo de dato:", data.dtype)

# Por ejemplo, inspeccionar un corte
import matplotlib.pyplot as plt
slice_index = data.shape[2] // 2
plt.imshow(data[:, :, slice_index], cmap="gray")
plt.title(f"Corte z = {slice_index}")
plt.colorbar()
plt.show()


In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd

# 1. Cargar archivo NIfTI
img = nib.load("mi_imagen.nii.gz")   # o .nii
data = img.get_fdata()               # numpy array (x, y, z)

print("Forma de la imagen:", data.shape)

# 2. Obtener dimensiones
nx, ny, nz = data.shape

# 3. Generar coordenadas (x, y, z)
x, y, z = np.meshgrid(
    np.arange(nx),
    np.arange(ny),
    np.arange(nz),
    indexing="ij"
)

# 4. Aplanar todo
df = pd.DataFrame({
    "x": x.flatten(),
    "y": y.flatten(),
    "z": z.flatten(),
    "intensidad": data.flatten()
})

# 5. Guardar a CSV
df.to_csv("imagen_nifti.csv", index=False)

print("✅ Archivo CSV creado con columnas (x, y, z, intensidad)")


# EEG 



Cómo se obtiene conectividad en EEG

El EEG mide actividad eléctrica en el tiempo en cada canal/electrodo. Para estimar conectividad necesitas calcular alguna métrica de dependencia entre las señales de distintos electrodos o regiones.

### Principales enfoques:

1. Conectividad funcional (sin direccionalidad):

    1. Correlación de Pearson → mide sincronía lineal entre pares de electrodos.

    1. Coherencia espectral → mide relación en el dominio de frecuencia.

    1. Phase Locking Value (PLV) → mide consistencia de fase entre señales.

    1. Mutual Information → mide dependencia no lineal.

1. Conectividad efectiva (con direccionalidad):

    1. Granger Causality → estima influencia temporal de una señal sobre otra.

    1. Transfer Entropy → mide flujos de información dirigidos.

1. Conectividad basada en grafos:

    1. Una vez que tienes la matriz (electrodos × electrodos), puedes aplicar teoría de grafos (medidas de grado, modularidad, eficiencia, hubs, etc.).

pip install mne

https://physionet.org/content/eegmat/1.0.0/

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# 1. Descargar y cargar datos EEG de ejemplo
# -------------------------
data_path = mne.datasets.sample.data_path()
raw_file = data_path + "/MEG/sample/sample_audvis_raw.fif"

raw = mne.io.read_raw_fif(raw_file, preload=True)

# Seleccionar solo EEG
raw.pick_types(meg=False, eeg=True, eog=False)

# -------------------------
# 2. Filtrar en banda alfa (opcional, 8–13 Hz)
# -------------------------
raw.filter(8, 13)

# -------------------------
# 3. Crear épocas de 2s de duración
# -------------------------
events = mne.make_fixed_length_events(raw, id=1, start=0, duration=2.0)
epochs = mne.Epochs(raw, events, event_id=1, tmin=0, tmax=2.0,
                    baseline=None, preload=True)

data = epochs.get_data()  # (n_epochs, n_channels, n_times)
print("Forma de los datos (épocas, canales, tiempo):", data.shape)

# -------------------------
# 4. Reorganizar datos para correlación
# -------------------------
n_epochs, n_channels, n_times = data.shape
# Concatenar todas las épocas en el tiempo
data_concat = data.transpose(1, 0, 2).reshape(n_channels, -1)  # (n_channels, n_epochs*n_times)

# -------------------------
# 5. Calcular correlación de Pearson entre canales
# -------------------------
corr_matrix = np.corrcoef(data_concat)

print("Matriz de conectividad (Pearson):", corr_matrix.shape)

# -------------------------
# 6. Visualizar la matriz
# -------------------------
plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar(label="Correlación de Pearson")
plt.title("Matriz de Conectividad EEG (Correlación de Pearson)")
plt.xlabel("Canales EEG")
plt.ylabel("Canales EEG")
plt.show()


In [ ]:
import mne

# Ruta al archivo EDF
edf_file = "./data/S001R01.edf"

# Cargar datos EEG
raw = mne.io.read_raw_edf(edf_file, preload=True)
print(raw.info)

# Ver los nombres de los canales
print(raw.ch_names)

# Visualizar los primeros segundos
raw.plot(duration=10, n_channels=10)

# Filtrar en banda alfa (8-13 Hz)
raw.filter(8, 13)

# Seleccionar solo canales EEG (ignorando EOG, EMG si los hay)
raw.pick_types(eeg=True, eog=False)

# Dividir en ventanas de 2 segundos
events = mne.make_fixed_length_events(raw, id=1, start=0, duration=2.0)
epochs = mne.Epochs(raw, events, event_id=1, tmin=0, tmax=2.0, baseline=None, preload=True)

data = epochs.get_data()  # (n_epochs, n_channels, n_times)
print("Forma de los datos (épocas, canales, tiempo):", data.shape)

import numpy as np
import matplotlib.pyplot as plt

# Concatenar todas las épocas en el tiempo
n_epochs, n_channels, n_times = data.shape
data_concat = data.transpose(1, 0, 2).reshape(n_channels, -1)  # (n_channels, n_epochs*n_times)

# Calcular correlación de Pearson entre canales
corr_matrix = np.corrcoef(data_concat)

# Visualizar la matriz
plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar(label="Correlación de Pearson")
plt.title("Matriz de conectividad EEG (correlación)")
plt.xlabel("Canales EEG")
plt.ylabel("Canales EEG")
plt.show()



In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne.connectivity import spectral_connectivity

# -------------------------
# 1. Cargar EEG de ejemplo (EDF de PhysioNet)
# -------------------------
edf_file = "./data/S001R01.edf"  # Cambia la ruta a tu archivo
raw = mne.io.read_raw_edf(edf_file, preload=True)

# Seleccionar solo canales EEG
raw.pick_types(eeg=True, eog=False)

# Filtrar en banda alfa (8-13 Hz)
raw.filter(8, 13)

# -------------------------
# 2. Crear épocas de 2s
# -------------------------
events = mne.make_fixed_length_events(raw, id=1, start=0, duration=2.0)
epochs = mne.Epochs(raw, events, event_id=1, tmin=0, tmax=2.0,
                    baseline=None, preload=True)

data = epochs.get_data()  # (n_epochs, n_channels, n_times)
n_epochs, n_channels, n_times = data.shape

# -------------------------
# 3. Conectividad Pearson
# -------------------------
data_concat = data.transpose(1, 0, 2).reshape(n_channels, -1)
corr_pearson = np.corrcoef(data_concat)

# -------------------------
# 4. Conectividad Coherencia espectral (método 'coh')
# -------------------------
con_coh, freqs, times, n_epochs_out, n_tapers = spectral_connectivity(
    data, method='coh', mode='fourier',
    fmin=8, fmax=13, sfreq=raw.info['sfreq'], verbose=False
)
# Elegimos la primera frecuencia (casi banda alfa promedio)
coh_matrix = con_coh[:, :, 0]

# -------------------------
# 5. Conectividad PLV (Phase Locking Value)
# -------------------------
con_plv, freqs, times, n_epochs_out, n_tapers = spectral_connectivity(
    data, method='plv', mode='fourier',
    fmin=8, fmax=13, sfreq=raw.info['sfreq'], verbose=False
)
plv_matrix = con_plv[:, :, 0]

# -------------------------
# 6. Visualizar matrices
# -------------------------
matrices = [corr_pearson, coh_matrix, plv_matrix]
titles = ["Pearson", "Coherencia Alfa", "PLV"]

plt.figure(figsize=(18, 5))
for i, (matrix, title) in enumerate(zip(matrices, titles)):
    plt.subplot(1, 3, i+1)
    plt.imshow(matrix, cmap="coolwarm", vmin=-1, vmax=1)
    plt.colorbar(label="Valor de conectividad")
    plt.title(title)
    plt.xlabel("Canales EEG")
    plt.ylabel("Canales EEG")
plt.tight_layout()
plt.show()


| Método                        | Qué mide                                             | Rango típico | Simetría     | Comentario                                                                  |
| ----------------------------- | ---------------------------------------------------- | ------------ | ------------ | --------------------------------------------------------------------------- |
| **Correlación de Pearson**    | Dependencia lineal entre señales                     | -1 a 1       | Simétrica    | Muy simple, mide similitud global en el tiempo.                             |
| **Coherencia espectral**      | Relación de amplitud y fase en frecuencia específica | 0 a 1        | Simétrica    | Dependencia en una banda de frecuencia; muy usada en EEG.                   |
| **Phase Locking Value (PLV)** | Sincronización de fase entre señales                 | 0 a 1        | Simétrica    | Sensible a la fase, ignora amplitud. Muy útil para oscilaciones cerebrales. |
| **Granger Causality**         | Influencia temporal direccional (causalidad)         | Varía        | No simétrica | Detecta qué señal predice a otra; genera matriz dirigida.                   |
| **Mutual Information**        | Dependencia lineal o no lineal                       | ≥0           | Simétrica    | Captura relaciones más complejas que Pearson.                               |
| **Transfer Entropy**          | Influencia direccional no lineal                     | ≥0           | No simétrica | Similar a Granger pero captura relaciones no lineales.                      |


🔹 Implicaciones

Si usas coherencia, los valores reflejarán sintonía en frecuencias, no necesariamente correlación temporal directa.

Si usas PLV, obtendrás una matriz que muestra solo coherencia de fase, independiente de amplitud.

Si usas Granger o Transfer Entropy, obtendrás matrices dirigidas, donde la conectividad (i→j) no es igual a (j→i).

Solo la correlación de Pearson y algunas métricas de dependencia lineal simple te darán matrices muy similares en estructura, pero los números exactos difieren.